## Statoil Iceberg Classification

### Classify if an image is an iceberg or a ship
### Use Keras/Tensorflow CNN for training

In [1]:
import numpy as np
import pandas as pd
import keras
from matplotlib import pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau
import cv2
np.random.seed(100)

Using TensorFlow backend.


In [2]:
# Read data
train = pd.read_json("train.json")
test = pd.read_json("test.json")

In [3]:
def get_scaled_imgs(df):
    imgs = []
    
    for i, row in df.iterrows():
        #make 75x75 image
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = band_1 + band_2 # plus since log(x*y) = log(x) + log(y)
        
        # Rescale
        a = (band_1 - band_1.mean()) / (band_1.max() - band_1.min())
        b = (band_2 - band_2.mean()) / (band_2.max() - band_2.min())
        c = (band_3 - band_3.mean()) / (band_3.max() - band_3.min())

        imgs.append(np.dstack((a, b, c)))

    return np.array(imgs)

In [4]:
Xtrain = get_scaled_imgs(train)
Ytrain = np.array(train['is_iceberg'])

In [5]:
train.inc_angle = train.inc_angle.replace('na',0)
idx_tr = np.where(train.inc_angle>0)

In [7]:
len(Xtrain)

1604

In [6]:
Ytrain = Ytrain[idx_tr[0]]
Xtrain = Xtrain[idx_tr[0],...]

In [9]:
len(Xtrain)

1471

In [7]:
def get_more_images(imgs):
    
    more_images = []
    vert_flip_imgs = []
    hori_flip_imgs = []
      
    for i in range(0,imgs.shape[0]):
        a=imgs[i,:,:,0]
        b=imgs[i,:,:,1]
        c=imgs[i,:,:,2]
        
        av=cv2.flip(a,1)
        ah=cv2.flip(a,0)
        bv=cv2.flip(b,1)
        bh=cv2.flip(b,0)
        cv=cv2.flip(c,1)
        ch=cv2.flip(c,0)
        
        vert_flip_imgs.append(np.dstack((av, bv, cv)))
        hori_flip_imgs.append(np.dstack((ah, bh, ch)))
      
    v = np.array(vert_flip_imgs)
    h = np.array(hori_flip_imgs)
       
    more_images = np.concatenate((imgs,v,h))
    
    return more_images

In [8]:
Xtr_more = get_more_images(Xtrain) 
Ytr_more = np.concatenate((Ytrain,Ytrain,Ytrain))

In [17]:
#define our model
def getModel():
    #Building the model
    gmodel=Sequential()
    #Conv Layer 1
    gmodel.add(Conv2D(64, kernel_size=(4, 4),activation='relu', input_shape=(75, 75, 3)))
    gmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    gmodel.add(Dropout(0.3))

    #Conv Layer 2
    gmodel.add(Conv2D(256, kernel_size=(4, 4), activation='relu' ))
    gmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    gmodel.add(Dropout(0.3))

    #Conv Layer 3
    gmodel.add(Conv2D(256, kernel_size=(4, 4), activation='relu'))
    gmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    gmodel.add(Dropout(0.3))

    #Conv Layer 4
    gmodel.add(Conv2D(64, kernel_size=(4, 4), activation='relu'))
    gmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    gmodel.add(Dropout(0.2))
    
    #Flatten the data for upcoming dense layers
    gmodel.add(Flatten())

    #Dense Layers
    gmodel.add(Dense(128))
    gmodel.add(Activation('relu'))
    gmodel.add(Dropout(0.3))

    #Dense Layer 2
    gmodel.add(Dense(64))
    gmodel.add(Activation('relu'))
    gmodel.add(Dropout(0.3))

    #Sigmoid Layer
    gmodel.add(Dense(1))
    gmodel.add(Activation('sigmoid'))

    mypotim=Adam(lr=0.005, beta_1=0.9, beta_2=0.99, epsilon=1e-08, decay=0.05)
    gmodel.compile(loss='binary_crossentropy',
                  optimizer=mypotim,
                  metrics=['accuracy'])
    gmodel.summary()
    return gmodel


def get_callbacks(filepath, patience=2):
    es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [es, msave]
file_path = ".model_weights.hdf5"
callbacks = get_callbacks(filepath=file_path, patience=5)

In [18]:
model = getModel()
model.summary()

batch_size = 32
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('.mdl_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, epsilon=1e-4, mode='min')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 72, 72, 64)        3136      
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 36, 36, 64)        0         
_________________________________________________________________
dropout_17 (Dropout)         (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 33, 33, 256)       262400    
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 16, 16, 256)       0         
_________________________________________________________________
dropout_18 (Dropout)         (None, 16, 16, 256)       0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 13, 13, 256)       1048832   
__________

In [19]:
model.fit(Xtr_more, Ytr_more, batch_size=batch_size, epochs=40, verbose=1, callbacks=[earlyStopping, mcp_save, reduce_lr_loss], validation_split=0.3)

Train on 3089 samples, validate on 1324 samples
Epoch 1/40

3089/3089 [==============================] - ETA: 0s - loss: 0.7194 - acc: 0.5646 - val_loss: 0.6466 - val_acc: 0.6715
Epoch 2/40

3089/3089 [==============================] - ETA: 0s - loss: 0.5602 - acc: 0.7601 - val_loss: 0.4905 - val_acc: 0.7749
Epoch 3/40

3089/3089 [==============================] - ETA: 0s - loss: 0.4575 - acc: 0.7993 - val_loss: 0.4710 - val_acc: 0.7878
Epoch 4/40

3089/3089 [==============================] - ETA: 0s - loss: 0.4239 - acc: 0.8077 - val_loss: 0.4174 - val_acc: 0.8165
Epoch 5/40

3089/3089 [==============================] - ETA: 0s - loss: 0.3985 - acc: 0.8203 - val_loss: 0.4013 - val_acc: 0.8278
Epoch 6/40

3089/3089 [==============================] - ETA: 0s - loss: 0.3745 - acc: 0.8391 - val_loss: 0.3738 - val_acc: 0.8316
Epoch 7/40

3089/3089 [==============================] - ETA: 0s - loss: 0.3545 - acc: 0.8420 - val_loss: 0.3502 - val_acc: 0.8384
Epoch 8/40

3089/3089 [============

In [20]:
model.load_weights(filepath = '.mdl_wts.hdf5')

score = model.evaluate(Xtrain, Ytrain, verbose=1)
print('Train score:', score[0])
print('Train accuracy:', score[1])


1471/1471 [==============================] - ETA: 0s
Train score: 0.238561381893
Train accuracy: 0.893269883865


In [21]:
test = pd.read_json('test.json')
test.inc_angle = test.inc_angle.replace('na',0)
Xtest = (get_scaled_imgs(test))
pred_test = model.predict(Xtest)

submission = pd.DataFrame({'id': test["id"], 'is_iceberg': pred_test.reshape((pred_test.shape[0]))})
print(submission.head(10))

submission.to_csv('./submission/submission_3.csv', index=False)

         id  is_iceberg
0  5941774d    0.608743
1  4023181e    0.799531
2  b20200e4    0.007306
3  e7f018bb    0.993923
4  4371c8c3    0.772024
5  a8d9b1fd    0.913940
6  29e7727e    0.251221
7  92a51ffb    0.988900
8  c769ac97    0.003941
9  aee0547d    0.000555
